# SkOpt Experiments and Data Processing

## Imports

In [2]:
cd C:\Users\hannag01\Documents\wsm-calibration\

C:\Users\hannag01\Documents\wsm-calibration


c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:

# Third party imports
import numpy as np
import matplotlib.pyplot as plt
from skopt.plots import plot_gaussian_process
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from skopt import gp_minimize
import pandas as pd
from scipy.stats import fisher_exact, norm, pearsonr

from ax.modelbridge.cross_validation import _gen_train_test_split

# Surrogate model code
from src.problems.surrogate import MultiFolderSurrogateProblem

## Setting up the surrogate model to optimise

In [4]:
# Load highlands surrogate problem
folder_path = r"C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Benchmarks\Highland-2024-07-16_11_*"
# Initialise the objective function
problem_args = {
         'folder': folder_path,
         'inputs': ['dd-los', 'gw-non-covid-los', 'syswatch-scale'],
         'outputs': ['emergency-output', 'delay-output'],
         'output_actuals_names': {'emergency-output':'emergency', 'delay-output':'delays'}
    }
p = MultiFolderSurrogateProblem(problem_args, True)

C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Benchmarks\Highland-2024-07-16_11_*


## Setup for optimisation loop

In [5]:
# Copy of the GP definition used in the current auto-calibration repositry
def create_gp(n_dims=3):
    '''Create a GP object'''
    cov_amplitude = ConstantKernel(1.0, (0.01, 1e8))
    matern_kernel = Matern(
        length_scale=np.ones(n_dims),
        length_scale_bounds=[(0.01, 1e8)]*n_dims, nu=2.5)

    kernel = cov_amplitude * matern_kernel + WhiteKernel(noise_level_bounds=(1e-10, 1e2))
    gp = GaussianProcessRegressor(
        kernel,
        n_restarts_optimizer=5,
        optimizer='fmin_l_bfgs_b',
        normalize_y=True
    )
    return gp


In [16]:
def evaluate(parameterization):
    """Function to convert the inputs and outputs needed by ax optimisation to/from the gp_minimise"""
    params_as_dict = dict(zip(p.inputs, parameterization))
    results = p.evaluate(params_as_dict)
    return results['error']

In [7]:
# Optimisation loop for the surrogate model. This is much faster to generate trials than the Ax version so will do these loops in memory

results = []
for i in range(30):
    gp = create_gp()
    res = gp_minimize(
        evaluate,
        [(param['bounds'][0], param['bounds'][1]) for param in p.parameters],
        base_estimator=gp,
        acq_func='EI',
        n_calls=40,
        verbose=True,
        xi=0.01,
        n_initial_points = 6
    )
    trials_df = pd.DataFrame({'error': res['func_vals']})
    trials_df['min_error'] = trials_df['error'].cummin()
    results.append({'trace': trials_df, 'loop_output': res})

print(len(results))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 247.5896
Current minimum: 247.5896
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0062
Function value obtained: 822.8455
Current minimum: 247.5896
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 2944.7853
Current minimum: 247.5896
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 117.7514
Current minimum: 117.7514
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 353.7546
Current minimum: 117.7514
Iteration No: 

## Process results

In [8]:
def mean_and_sem_of_trials(dfs, metric):
    """ Combines a metric from multiple runs with the same parameters in order to find the mean and standard error of that metric
    Arguments:
        dfs: iterable of pandas dataframes where each row is one of the trials, and contains a column with a metric to track over the course of optimisation
        metric: name of column to track mean and sem for
    Returns:
        df: a single dataframe with the trial index as the index and columns for the means and errors over all the runs passed

    """
    combined_dfs = pd.concat(dfs, axis=1)
    # Optimisation runs with incomplete data will be dropped
    combined_dfs = combined_dfs[[metric]].dropna(axis=1,how='any')
    mean = combined_dfs.mean(axis=1)
    sem = combined_dfs.sem(axis=1)
    count = combined_dfs.count(axis=1)
    positive_error = mean + sem
    negative_error = mean - sem
    return pd.DataFrame({'mean': mean, 'sem':sem, 'positive_error': positive_error, 'negative_error':negative_error, 'count': count})


In [9]:
# Create an output file to use with the ax data
trace_dfs = [r['trace'] for r in results]
print(len(trace_dfs))
combined_trace = mean_and_sem_of_trials(trace_dfs, 'min_error')
combined_trace.to_csv(r'C:\Users\hannag01\Documents\parallel-bayesopt-experiments\data\skopt_agg_trace_highlands_surrogate_2024-07-16_11_2.csv')

30


## Gen times

Estimates of the SkOpt trial generation time

In [10]:
# Run the same number of evaluations again to subtract from the calibration time
results_evaluations = []
for result in results:
    for params in result['loop_output']['x_iters']:
        results_evaluations.append(evaluate(params))
len(results_evaluations)

1200

In [11]:
# Based on the runtime for the cells
time_for_30_calibrations = 7 * 60 + 45
num_trials = 40 * 30
local_run_time_for_30 = 0.8

gen_time_per_trial = (time_for_30_calibrations - local_run_time_for_30) / num_trials
gen_time_per_trial

0.3868333333333333

## Cross Validation of GP

In [12]:
def cross_validate_res(results):
    X = np.array(results['x_iters'])
    y = results['func_vals']

    n = len(y)
    arm_names  = np.array(range(n))

    predicted = []
    observed = []
    for train_idxs, test_idxs in  _gen_train_test_split(n, arm_names=arm_names):
        gp = create_gp(n_dims=3)
        train_X = X[list(train_idxs)]
        train_y = y[list(train_idxs)]
        test_X = X[list(test_idxs)]
        test_y = y[list(test_idxs)]

        fitted_gp = gp.fit(train_X, train_y)
        prediction = fitted_gp.predict(test_X)
        predicted.append(prediction)
        observed.append(test_y)
    return predicted, observed
    

In [13]:
correlation_cos = []
for r in results:
    p,o = cross_validate_res(r['loop_output'])
    correlation_cos.append(pearsonr(np.array(p).flatten(), np.array(o).flatten()))

c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase 

In [14]:
pd.DataFrame({'correlation_coefficient': [c.statistic for c in correlation_cos], 'p_value': [c.pvalue for c in correlation_cos]}).to_csv((r'C:\Users\hannag01\Documents\parallel-bayesopt-experiments\data\skopt_highlands_surrogate_2024-07-14_11_crossval_coefficient.csv'))

## Grampian Dataset

In [15]:
# Load Grampian surrogate problem
folder_path = r"C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Benchmarks\GRAMPIAN-20240612_11_*"
# Initialise the objective function
problem_args = {
         'folder': folder_path,
         'inputs': ['dd-los', 'gw-non-covid-los', 'syswatch-scale'],
         'outputs': ['emergency-output', 'delay-output'],
         'output_actuals_names': {'emergency-output':'emergency', 'delay-output':'delays'}
    }
p = MultiFolderSurrogateProblem(problem_args, True)

C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Benchmarks\GRAMPIAN-20240612_11_*


In [17]:
# Optimisation loop

results = []
for i in range(30):
    gp = create_gp()
    res = gp_minimize(
        evaluate,
        [(param['bounds'][0], param['bounds'][1]) for param in p.parameters],
        base_estimator=gp,
        acq_func='EI',
        n_calls=40,
        verbose=True,
        xi=0.01,
        n_initial_points = 6
    )
    trials_df = pd.DataFrame({'error': res['func_vals']})
    trials_df['min_error'] = trials_df['error'].cummin()
    results.append({'trace': trials_df, 'loop_output': res})

print(len(results))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0281
Function value obtained: 278.1571
Current minimum: 278.1571
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0062
Function value obtained: 23.2317
Current minimum: 23.2317
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0012
Function value obtained: 1353.9097
Current minimum: 23.2317
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 50.3801
Current minimum: 23.2317
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 654.9930
Current minimum: 23.2317
Iteration No: 6 star

In [18]:
# Create an output file to use with the ax data
trace_dfs = [r['trace'] for r in results]
print(len(trace_dfs))
combined_trace = mean_and_sem_of_trials(trace_dfs, 'min_error')
combined_trace.to_csv(r'C:\Users\hannag01\Documents\parallel-bayesopt-experiments\data\skopt_agg_trace_grampian_surrogate.csv')

30


In [19]:
# Cross validate
correlation_cos = []
for r in results:
    p,o = cross_validate_res(r['loop_output'])
    correlation_cos.append(pearsonr(np.array(p).flatten(), np.array(o).flatten()))

c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or 

In [20]:
#Export
pd.DataFrame({'correlation_coefficient': [c.statistic for c in correlation_cos], 'p_value': [c.pvalue for c in correlation_cos]}).to_csv((r'C:\Users\hannag01\Documents\parallel-bayesopt-experiments\data\skopt_grampian_surrogate_crossval_coefficient.csv'))